# 3: Build a neural network with AutoKeras

In [4]:
from modules.utils import *
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.sparse
import os

# import keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential, Model 
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, GlobalMaxPool1D, Flatten, Input, concatenate, Dropout, Activation, regularizers, BatchNormalization


from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix

print('Autokeras needs TF>=2')
print('TF version:',tf.__version__)


Autokeras needs TF>=2
TF version: 2.4.0


In [23]:
import warnings
warnings.filterwarnings("ignore")


In [5]:
cwd = os.getcwd()
print(cwd)
os.chdir(cwd)

C:\Users\brizio\Documents\PythonNB\FICOchallenge


## Load prepped data
- Normal Scaling of numeric variables (prep_option = 1)
- Binning (following Rudin) and one hot encoding (prep_option = 2)
- Binning and applying WOE, calculating WOE on Rudin's bins (prep_option = 3)
- Binning and applying WOE, following Rudin (prep_option = 4)

In [6]:
data, X1, y, y_onehot = f_load_data(1)
X_train, X_test, y_train, y_test = train_test_split(X1, y_onehot, test_size = .25, random_state = 2020, shuffle = True)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X1.head()

Normal Scaling of numeric variables (prep_option = 1)
Target: Bad (y=1)
Bad     5459
Good    5000
Name: RiskPerformance, dtype: int64
[[   0 5000]
 [   1 5459]]
X shape: (10459, 23)
(7844, 23) (2615, 23) (7844, 1) (2615, 1)


,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,PercentInstallTrades,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance
0,-0.588324,-0.366575,-0.341585,0.261899,0.043983,1.176415,0.060341,-0.140837,-0.232309,-0.513377,...,0.538241,0.053626,-0.273077,-0.258498,0.045581,-1.120175,1.091102,0.005886,0.292417,0.249757
1,-0.304241,-1.150684,0.485414,-0.846892,-1.340237,1.574221,1.750041,0.513051,-0.671328,-1.312078,...,1.730629,0.053626,-0.273077,-0.258498,-1.052271,-1.120175,-0.721739,-2.210425,-2.387770,-2.240297
2,-0.020158,-1.077744,-0.266403,-1.285252,-0.801929,-0.017002,0.060341,0.513051,-0.671328,0.551559,...,0.587924,0.053626,0.985121,1.013949,0.710946,0.637562,0.184681,0.252143,0.292417,0.863249
3,-0.067506,-0.138636,-0.567130,-0.021745,0.659193,0.380804,0.482766,0.243803,3.377399,0.285325,...,1.233801,0.053626,1.299670,1.013949,1.343042,1.041366,0.637892,0.744657,0.888014,1.043688
4,0.642701,1.356643,1.387594,1.499621,-0.571226,-0.017002,0.060341,0.513051,-0.671328,0.551559,...,-0.356050,0.053626,0.041473,0.059614,0.644409,1.183886,-0.041924,0.005886,-0.005381,0.646722


### Onehot encoded data

In [31]:
data, X1, y, y_onehot = f_load_data(2)
X_train, X_test, y_train, y_test = train_test_split(X1, y_onehot, test_size = .25, random_state = 2020, shuffle = True)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X1.head()

Binning (following Rudin) and one hot encoding (prep_option = 2)
Target: Bad (y=1)
Bad     5459
Good    5000
Name: RiskPerformance, dtype: int64
[[   0 5000]
 [   1 5459]]
X shape: (10459, 130)
(7844, 130) (2615, 130) (7844, 1) (2615, 1)


,"ExternalRiskEstimate_bin_(-inf, -9.0]","ExternalRiskEstimate_bin_(-7.0, 63.0]","ExternalRiskEstimate_bin_(63.0, 70.0]","ExternalRiskEstimate_bin_(70.0, 75.0]","ExternalRiskEstimate_bin_(75.0, 80.0]","ExternalRiskEstimate_bin_(80.0, inf]","MSinceOldestTradeOpen_bin_(-inf, -9.0]","MSinceOldestTradeOpen_bin_(-9.0, -8.0]","MSinceOldestTradeOpen_bin_(-7.0, 91.0]","MSinceOldestTradeOpen_bin_(91.0, 134.0]",...,"NumBank2NatlTradesWHighUtilization_bin_(2.0, 3.0]","NumBank2NatlTradesWHighUtilization_bin_(3.0, 5.0]","NumBank2NatlTradesWHighUtilization_bin_(5.0, inf]","PercentTradesWBalance_bin_(-inf, -9.0]","PercentTradesWBalance_bin_(-9.0, -8.0]","PercentTradesWBalance_bin_(-7.0, 47.0]","PercentTradesWBalance_bin_(47.0, 66.0]","PercentTradesWBalance_bin_(66.0, 73.0]","PercentTradesWBalance_bin_(73.0, 86.0]","PercentTradesWBalance_bin_(86.0, inf]"
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [32]:
# basic data preparation
from sklearn.preprocessing import LabelEncoder
X = X1.astype('float32')
y = LabelEncoder().fit_transform(y_onehot['Bad'].astype('int8'))

X_onehot_train, X_onehot_test, y_onehot_train, y_onehot_test, = \
train_test_split(X, y, test_size = .25, random_state = 2020, shuffle = True)
print(X_onehot_train.shape, X_onehot_test.shape, y_onehot_train.shape, y_onehot_test.shape)


from autokeras import StructuredDataClassifier
# define the search
search = StructuredDataClassifier(max_trials=20, project_name="AKencodedData")
# perform the search
search.fit(x=X_onehot_train, y=y_onehot_train, verbose=1)
# evaluate the model
# loss, acc = search.evaluate(X_onehot_test, y_onehot_test, verbose=1)

Trial 20 Complete [00h 00m 28s]
val_accuracy: 0.6519480347633362

Best val_accuracy So Far: 0.732467532157898
Total elapsed time: 00h 17m 17s
INFO:tensorflow:Oracle triggered exit
Epoch 1/6
246/246 [==============================] - 2s 3ms/step - loss: 0.7576 - accuracy: 0.5521
Epoch 2/6
246/246 [==============================] - 1s 3ms/step - loss: 0.6241 - accuracy: 0.6785
Epoch 3/6
246/246 [==============================] - 1s 3ms/step - loss: 0.5892 - accuracy: 0.6990
Epoch 4/6
246/246 [==============================] - 1s 3ms/step - loss: 0.5839 - accuracy: 0.7072
Epoch 5/6
246/246 [==============================] - 1s 3ms/step - loss: 0.5730 - accuracy: 0.7200
Epoch 6/6
246/246 [==============================] - 1s 3ms/step - loss: 0.5572 - accuracy: 0.7312
INFO:tensorflow:Assets written to: .\AKencodedData\best_model\assets


In [24]:
search.evaluate(X_onehot_test, y_onehot_test, verbose=1);

82/82 [==============================] - 0s 1ms/step - loss: 0.5891 - accuracy: 0.6841


In [33]:
# get the best performing model
model = search.export_model()
# summarize the loaded model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 130)]             0         
_________________________________________________________________
multi_category_encoding (Mul (None, 130)               0         
_________________________________________________________________
normalization (Normalization (None, 130)               261       
_________________________________________________________________
dense (Dense)                (None, 32)                4192      
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dropout (Dropout)            (None, 32)                0     

In [34]:
# save the best performing model to file
model.save('./results/model_encoded')

INFO:tensorflow:Assets written to: ./results/model_encoded\assets


In [35]:
# scores = model.evaluate(X_onehot_train.values, y_onehot_train)
scores = model.evaluate(X_onehot_train, y_onehot_train, verbose=0)
print("Training Set:", "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# scores = model.evaluate(X_onehot_test.values, y_onehot_test)
scores = model.evaluate(X_onehot_test, y_onehot_test, verbose = 0)
print("Test Set:", "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# confusion_matrix(y_onehot_test, model.predict_classes(X_onehot_test), sample_weight=None)

Training Set: 
accuracy: 74.01%
Test Set: 
accuracy: 71.74%


### WoE variables

In [25]:
data, X1, y, y_onehot = f_load_data(4)
X_train, X_test, y_train, y_test = train_test_split(X1, y_onehot, test_size = .25, random_state = 2020, shuffle = True)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X1.head()

Binning and applying WOE, following Rudin (prep_option = 4)
Target: Bad (y=1)
Bad     5459
Good    5000
Name: RiskPerformance, dtype: int64
[[   0 5000]
 [   1 5459]]
X shape: (10459, 23)
(7844, 23) (2615, 23) (7844, 1) (2615, 1)


,ExternalRiskEstimate_bin_WOE,MSinceOldestTradeOpen_bin_WOE,MSinceMostRecentTradeOpen_bin_WOE,AverageMInFile_bin_WOE,NumSatisfactoryTrades_bin_WOE,NumTrades60Ever2DerogPubRec_bin_WOE,NumTrades90Ever2DerogPubRec_bin_WOE,NumTotalTrades_bin_WOE,NumTradesOpeninLast12M_bin_WOE,PercentTradesNeverDelq_bin_WOE,...,PercentInstallTrades_bin_WOE,NetFractionInstallBurden_bin_WOE,NumInstallTradesWBalance_bin_WOE,MSinceMostRecentInqexcl7days_bin_WOE,NumInqLast6M_bin_WOE,NumInqLast6Mexcl7days_bin_WOE,NetFractionRevolvingBurden_bin_WOE,NumRevolvingTradesWBalance_bin_WOE,NumBank2NatlTradesWHighUtilization_bin_WOE,PercentTradesWBalance_bin_WOE
0,1.799,0.086,0.083,0.269,0.166,0.952,-0.021,-0.097,-0.021,1.012,...,-0.503,0.047,0.242,1.223,-0.047,-0.051,-0.088,0.034,-0.601,-0.130
1,1.799,0.549,0.083,1.238,1.999,0.952,-0.053,0.535,-0.021,-0.147,...,0.161,0.047,0.256,1.223,-0.047,-0.051,-0.739,-0.188,0.601,-0.982
2,1.017,0.549,0.083,1.238,0.539,-0.021,-0.021,0.535,0.428,-0.147,...,-0.503,0.147,0.242,1.223,0.170,0.021,0.633,-0.263,-0.601,0.203
3,1.017,0.086,0.083,0.269,-0.086,-0.021,-0.021,-0.377,0.287,0.366,...,-0.145,0.363,0.313,1.223,0.471,0.021,0.633,-0.150,0.541,0.772
4,-1.094,-0.148,0.000,0.000,0.539,-0.021,-0.021,0.116,-0.021,-0.147,...,-0.620,0.363,0.242,1.223,-0.047,-0.051,0.633,-0.188,-0.601,0.203


In [27]:
# basic data preparation
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import preprocessing
X = X1.astype('float32')
y = LabelEncoder().fit_transform(y_onehot['Bad'].astype('int8'))

X_onehot_train, X_onehot_test, y_onehot_train, y_onehot_test, = \
train_test_split(X, y, test_size = .25, random_state = 2020, shuffle = True)
print(X_onehot_train.shape, X_onehot_test.shape, y_onehot_train.shape, y_onehot_test.shape)


from autokeras import StructuredDataClassifier
# define the search
search = StructuredDataClassifier(max_trials=20, project_name="AKWOEdata")
# perform the search
search.fit(x=X_onehot_train, y=y_onehot_train, verbose=1)
# evaluate the model
# loss, acc = search.evaluate(X_onehot_test, y_onehot_test, verbose=1)

Trial 20 Complete [00h 00m 16s]
val_accuracy: 0.7110389471054077

Best val_accuracy So Far: 0.7207792401313782
Total elapsed time: 00h 03m 53s
INFO:tensorflow:Oracle triggered exit
246/246 [==============================] - 1s 2ms/step - loss: 0.6344 - accuracy: 0.6447
INFO:tensorflow:Assets written to: .\AKWOEdata\best_model\assets


In [28]:
search.evaluate(X_onehot_test, y_onehot_test, verbose=1);

82/82 [==============================] - 0s 1ms/step - loss: 0.5822 - accuracy: 0.6945


In [29]:
# get the best performing model
model = search.export_model()
# summarize the loaded model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 23)                0         
_________________________________________________________________
normalization (Normalization (None, 23)                47        
_________________________________________________________________
dense (Dense)                (None, 32)                768       
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)                0     

In [40]:
# save the best performing model to file
model.save('model_WOE_auto',save_format='tf')

INFO:tensorflow:Assets written to: model_WOE_auto\assets


In [30]:
# scores = model.evaluate(X_onehot_train.values, y_onehot_train)
scores = model.evaluate(X_onehot_train, y_onehot_train, verbose=0)
print("Training Set:", "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# scores = model.evaluate(X_onehot_test.values, y_onehot_test)
scores = model.evaluate(X_onehot_test, y_onehot_test, verbose = 0)
print("Test Set:", "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# confusion_matrix(y_onehot_test, model.predict_classes(X_onehot_test), sample_weight=None)

Training Set: 
accuracy: 71.51%
Test Set: 
accuracy: 69.45%
